## LSA(잠재 의미 분석)
- 문서 안에서 단어 사이의 잠재적인 의미 구조를 추출하는 기법 
- TF-IDF 방식은 단어 간의 의미적 유사성을 반영X
- TF-IDF 에서 SVD 분해를 하여 단어 간의 의미를 파악
- TF-IDF에서 차원 축소( PCA, t-SNE과 같은 축소 기법 중 하나를 사용 )하여 관계성을 확인
- LSA 효과 
    - 백터 공간의 차원을 줄여서 계산 효율을 증가 (차원 축소)
    - '영화', '필름' 비슷한 문맥의 단어를 가까운 백터로 이동(의미 유추) 
    - 문서들을 주제별로 분류 기능(토픽 분석)

- TruncatedSVD(차원 축소 모델)
    - 절단된 특이값의 분해
    - 고차원 희소 행렬(값이 0인 행렬)을 낮은 차원으로 압축하여 데이터 구조적 의미를 유지 
    - 자연어 처리, 추천 시스템, 의미 분석, 잠재적인 토픽 분석 주로 사용

    - TF-IDF 행렬은 우선은 고차원 -> 저차원
    - 0으로 이루어진 희소행렬들을 구조적인 의미를 유지하면서 값들을 부여 
    - 같은 토픽의 문서는 같은 백터 공간에서 가깝게 위치 -> 유사도 기반 자연어 처리에 활용
    

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
from konlpy.tag import Okt

In [ ]:
# 토큰화 함수를 정의 -> pos 필터 
okt = Okt()

def tokenize(text):
    result = [ word for word, pos in okt.pos(text) 
              if pos in ['Noun', 'Adjective', 'Verb'] ]
    return result

In [ ]:
docs = [
    '이 영화 정말 재미있었다', 
    '매우 연기가 뛰어나다', 
    '이 영화 별로다', 
    '지루한 영화는 보기 어렵다', 
    '정말 훌륭한 연기였다',
    "연기가 별로라서 지루했다"
]

In [ ]:
# TF-IDF 백터화 
tfidf = TfidfVectorizer(
    tokenizer=tokenize, 
    ngram_range=(1, 1), 
    min_df = 1, 
    max_df = 0.8, 
    sublinear_tf=True, 
    lowercase=False
)

In [ ]:
X_tfidf = tfidf.fit_transform(docs)

In [ ]:
X_tfidf.shape

In [ ]:
# SVD를 이용한 차원 축소 (LSA 적용)
lsa = TruncatedSVD(n_components=2, random_state=42)
X_lsa = lsa.fit_transform(X_tfidf)

In [ ]:
df_lsa = pd.DataFrame(X_lsa, columns = ['topic1', 'topic2'] )
df_lsa

In [ ]:
df_lsa['document'] = docs

In [ ]:
df_lsa

In [ ]:
terms = tfidf.get_feature_names_out()
components = lsa.components_

df_terms = pd.DataFrame(components.T, index = terms, 
                        columns = ['topic1', 'topic2'])
df_terms.sort_values('topic1')

1. ratings_test.txt 파일 로드 
2. 결측치제외, id 컬럼 제외
3. document의 중복된 데이터를 제거 
4. 상위 5000개를 필터 
5. 독립 변수, 종속변수 train, test 로 데이터 분할 
6. X_train을 이용하여 tdifd, LSA 작업 
7. 모델은 SVC 사용하여 학습 및 평가 

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("../data/ratings_test.txt", sep='\t')
df.dropna(inplace=True)
df.drop('id', axis=1, inplace=True)

In [ ]:
# documet 컬럼의 데이터 중 중복 데이터 제거 
df.drop_duplicates('document', inplace=True)
df.info()

In [ ]:
# 독립, 종속 변수 생성 
X = df['document'].values
Y = df['label'].values

In [ ]:
# train, test로 데이터셋 변경
X_tr, X_te, Y_tr, Y_te = train_test_split(
    X, Y, test_size=0.2, stratify=Y, random_state=42
)

In [ ]:
# train , test 데이터 개수를 줄여준다( 빠른 실행을 위해 )
X_tr = X_tr[:5000]
X_te = X_te[:1000]
Y_tr = Y_tr[:5000]
Y_te = Y_te[:1000]

In [ ]:
# 토큰화 함수를 생성 
okt = Okt()

def tokenize(text):
    return okt.morphs(text)
# 백터화
vector = TfidfVectorizer(
    tokenizer=tokenize, 
    lowercase= False, 
    ngram_range=(1,2), 
    min_df= 3
)

In [ ]:
# train 데이터를 이용하여 백터화 작업 
# X_tr를 이용하여 vector에 학습
vector.fit(X_tr)

In [ ]:
X_tr_vc = vector.transform(X_tr)
# train 데이터로 학습한 변환 모델에 test데이터로 변환 
X_te_vc = vector.transform(X_te)

In [ ]:
svc = SVC(
    kernel='linear', 
    C = 1.0, 
    random_state=42
)

In [ ]:
# tf_idf 변환을 한 데이터를 이용하여 SVC 모델에 학습 및 평가 
svc.fit(X_tr_vc, Y_tr)

In [ ]:
# test데이터를 이용하여 예측
pred_vc = svc.predict(X_te_vc)

In [ ]:
# 정확도, f1 score 확인 
acc_vc = accuracy_score(pred_vc, Y_te)
f1_vc = f1_score(pred_vc, Y_te)
print(f"TD-IDF 변환 후 학습 정확도 : {round(acc_vc, 4)} f1 : {round(f1_vc, 4)}")

In [ ]:
X_tr_vc.shape

In [ ]:
# LSA 정의 
lsa = TruncatedSVD(
    n_components= 200, 
    random_state= 42
)
# lsa를 이용하여 학습 
lsa.fit(X_tr_vc)

In [ ]:
X_tr_lsa = lsa.transform(X_tr_vc)
X_te_lsa = lsa.transform(X_te_vc) 

In [ ]:
svc.fit(X_tr_lsa, Y_tr)

In [ ]:
pred_svc = svc.predict(X_te_lsa)

In [ ]:
acc_lsa = accuracy_score(pred_svc, Y_te)
f1_lsa = f1_score(pred_svc, Y_te)

print(f'LSA 작업 후 정확도 : {round(acc_lsa, 4)}, f1 : {round(f1_lsa, 4)}')

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
std = StandardScaler(with_mean= True)

In [ ]:
X_tr_std = std.fit_transform(X_tr_lsa)
X_te_std = std.transform(X_te_lsa)

In [ ]:
svc.fit(X_tr_std, Y_tr)

In [ ]:
pred_std = svc.predict(X_te_std)

In [46]:
acc_std = accuracy_score(pred_std, Y_te)
f1_std = f1_score(pred_std, Y_te)

print(f'TD-IDF -> LSA -> Scaler 작업 후 정확도 : {round(acc_std, 4)} f1 : {round(f1_std, 4)}')

TD-IDF -> LSA -> Scaler 작업 후 정확도 : 0.745 f1 : 0.7341
